In [7]:
import numpy as np
from pathlib import Path
import pandas as pd
import os

from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting

from pymoo.config import Config

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Config.warnings["not_compiled"] = False

In [8]:
DISTANCE_KM = 20
RAW_DIR = Path(os.getcwd()).parent / "data" / "raw"
INTERIM_DIR = Path(os.getcwd()).parent / "data" / "interim"

PARETO_PATH = INTERIM_DIR / "preproc_pareto_data.pkl"
EACH_PARETO_PATH = RAW_DIR / f"pareto_{int(DISTANCE_KM)}km.pkl"

# Read data


In [9]:
# Convert to DataFrame
columns = [
    "accel_ms2",
    "decel_ms2",
    "time_min",
    "energy_kwh",
]
pareto_df = pd.read_pickle(PARETO_PATH)
one_pareto_pkl = pd.read_pickle(EACH_PARETO_PATH)
one_pareto_df = pd.DataFrame(one_pareto_pkl.get("pareto_front"), columns=columns)

pareto_df.head()

,accel_ms2,decel_ms2,time_min,energy_kwh,distance_km,max_speed_mps,max_bettery_kwh,time_weight,energy_weight
0,0.213321,0.209042,17.447130,1.565973,20,20.0,10.0,0.006693,0.993307
1,1.999999,1.999592,16.749167,1.594058,20,20.0,10.0,0.993307,0.006693
2,1.226421,1.205027,16.801731,1.591707,20,20.0,10.0,0.989520,0.010480
3,1.980324,1.979324,16.749995,1.591878,20,20.0,10.0,0.990028,0.009972
4,0.220566,0.213304,17.421467,1.567303,20,20.0,10.0,0.007773,0.992227


## Analysis

In [10]:
# Calculate quantiles
filtered_pareto_df = pareto_df.query("distance_km == @DISTANCE_KM")
quantiles = filtered_pareto_df[["time_weight", "energy_weight"]].quantile(
    q=[0.25, 0.5, 0.75]
)

# Create subplot grid with 2 rows and 2 columns
fig = make_subplots(
    rows=2,
    cols=2,
    vertical_spacing=0.15,
    horizontal_spacing=0.15,
    subplot_titles=(
        "Time Weight Distribution",
        "Energy Weight Distribution",
        "Time Weight Spread",
        "Energy Weight Spread",
    ),
    row_heights=[0.7, 0.3],  # Adjust vertical ratio
)

# Add histograms to top row
fig.add_trace(
    go.Histogram(
        x=filtered_pareto_df["time_weight"],
        name="Time Weight",
        marker_color="#1f77b4",
        nbinsx=30,
        showlegend=True,
        marker=dict(line=dict(width=1, color="white")),
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Histogram(
        x=filtered_pareto_df["energy_weight"],
        name="Energy Weight",
        marker_color="#ff7f0e",
        nbinsx=30,
        showlegend=True,
        marker=dict(line=dict(width=1, color="white")),
    ),
    row=1,
    col=2,
)

# Add box plots to bottom row
fig.add_trace(
    go.Box(
        x=filtered_pareto_df["time_weight"],
        name="Time Weight",
        boxpoints="all",
        jitter=0.3,
        pointpos=-1.8,
        line_color="#1f77b4",
        quartilemethod="linear",
        showlegend=False,
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Box(
        x=filtered_pareto_df["energy_weight"],
        name="Energy Weight",
        boxpoints="all",
        jitter=0.3,
        pointpos=1.8,
        line_color="#ff7f0e",
        quartilemethod="linear",
        showlegend=False,
    ),
    row=2,
    col=2,
)

# Add quartile lines to box plots
for q in [0.25, 0.5, 0.75]:
    fig.add_vline(
        x=quantiles.loc[q, "time_weight"],
        line_dash="dot",
        line_color="#1f77b4",
        row=2,
        col=1,
        annotation_text=f"Q{q * 100:.0f}",
        annotation_position="bottom right",
    )

    fig.add_vline(
        x=quantiles.loc[q, "energy_weight"],
        line_dash="dot",
        line_color="#ff7f0e",
        row=2,
        col=2,
        annotation_text=f"Q{q * 100:.0f}",
        annotation_position="bottom right",
    )

# Update axis labels and styling
fig.update_xaxes(title_text="Weight Value", row=1, col=1)
fig.update_xaxes(title_text="Weight Value", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Weight Value", row=2, col=1)
fig.update_yaxes(title_text="Weight Value", row=2, col=2)
fig.update_xaxes(showticklabels=False, row=2, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=2)

# Final layout adjustments
fig.update_layout(
    title_text="Weight Distribution Analysis: Time vs Energy Parameters",
    template="plotly_white",
    height=800,
    width=1200,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

# Custom hover information
fig.update_traces(
    selector=dict(type="box"),
    hoverinfo="x",
    hovertemplate="<b>%{fullData.name}</b><br>Value: %{x:.4f}<extra></extra>",
)

fig.update_traces(
    selector=dict(type="histogram"),
    hovertemplate="<b>%{fullData.name}</b><br>Value: %{x}<br>Count: %{y}<extra></extra>",
)

# Save and show
fig.show()
fig.write_image(
    "../reports/figures/weight_distribution_analysis.png",
    width=1200,
    height=800,
    scale=2,
    engine="kaleido",
)

## Pareto Front Interpretation
The optimal solutions represent trade-offs:
- **Left Extreme**: Minimum time (aggressive acceleration)
- **Right Extreme**: Minimum energy (gentle acceleration)
- **Middle**: Balanced compromises

The Pareto front $ \mathcal{X}_{\text{Pareto}} $ consists of non-dominated solutions:
\begin{aligned}
\mathcal{X}_{\text{Pareto}} = \left\{ \mathbf{J}_i = [T_i, E_i]^T \, \big| \, \nexists \, \mathbf{J}_j \text{ where } T_j \leq T_i \text{ and } E_j \leq E_i \right\}
\end{aligned}

### 1. Visualize problem bounds

In [11]:
# Sort the Pareto front by travel time (x-axis)
sorted_front = pareto_df.sort_values(by="time_min")

# Create subplots with 1 row and 2 columns
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Design Space", "Pareto Front"),
    horizontal_spacing=0.12,
    column_widths=[0.3, 0.3],
)

# 1. Design Space Plot (Left)
fig.add_trace(
    go.Scatter(
        x=pareto_df["accel_ms2"],
        y=pareto_df["decel_ms2"],
        mode="markers",
        marker=dict(
            size=8,
            color="rgba(255, 100, 100, 0.7)",
            line=dict(width=1, color="DarkRed"),
        ),
        name="Design Points",
        hovertemplate="<b>Accel Phase 1</b>: %{x:.2f} m/s²<br><b>Accel Phase 2</b>: %{y:.2f} m/s²<extra></extra>",
    ),
    row=1,
    col=1,
)

# 2. Pareto Front Plot (Right)
fig.add_trace(
    go.Scatter(
        x=pareto_df["time_min"],
        y=pareto_df["energy_kwh"],
        mode="markers",
        marker=dict(
            size=8,
            color="rgba(50, 200, 50, 0.8)",
            opacity=0.5,
            line=dict(width=1, color="DarkGreen"),
        ),
        name="Pareto Front",
        hovertemplate="<b>Time</b>: %{x:.2f} min<br><b>Energy</b>: %{y:.4f} kWh<extra></extra>",
    ),
    row=1,
    col=2,
)

# Add smooth trend line
fig.add_trace(
    go.Scatter(
        x=sorted_front["time_min"],
        y=sorted_front["energy_kwh"],
        mode="lines",
        line=dict(color="black", width=3, shape="spline", smoothing=0.5),
        name="Pareto Frontier",
        hoverinfo="skip",
        opacity=0.5,
    ),
    row=1,
    col=2,
)

# Update layout
fig.update_layout(
    title_text="EV Control Optimization Results",
    title_x=0.5,
    title_font=dict(size=20),
    showlegend=False,
    width=1400,
    height=550,
    margin=dict(l=60, r=60, b=80, t=100),
    plot_bgcolor="rgba(248,248,255,1)",
    paper_bgcolor="rgba(248,248,255,1)",
)

# Customize axes
for col in [1, 2]:
    fig.update_xaxes(
        showgrid=True,
        gridcolor="rgba(200,200,200,0.5)",
        zerolinecolor="lightgrey",
        row=1,
        col=col,
    )
    fig.update_yaxes(
        showgrid=True,
        gridcolor="rgba(200,200,200,0.5)",
        zerolinecolor="lightgrey",
        row=1,
        col=col,
    )

# Axis titles
fig.update_xaxes(title_text="Acceleration Phase 1 (m/s²)", row=1, col=1)
fig.update_yaxes(title_text="Acceleration Phase 2 (m/s²)", row=1, col=1)
fig.update_xaxes(title_text="Travel Time (min)", row=1, col=2)
fig.update_yaxes(title_text="Energy Consumption (kWh)", row=1, col=2)

# Add annotations
fig.add_annotation(
    x=0.15,
    y=0.02,
    xref="paper",
    yref="paper",
    text="● Design Parameters",
    showarrow=False,
    font=dict(color="darkred"),
)
fig.add_annotation(
    x=0.78,
    y=0.02,
    xref="paper",
    yref="paper",
    text="◆ Pareto Front",
    showarrow=False,
    font=dict(color="darkgreen"),
)

fig.show()
fig.write_image(
    "../reports/figures/pareto_front_design_space.png",
    width=1400,
    height=550,
    scale=2,
    engine="kaleido",
)

### 2. Visualize the Pareto Front


In [12]:
# ==============================================
# DATA PREPARATION & UNIT CONVERSION
# ==============================================

# Extract metadata from optimization results
optimization_metadata = one_pareto_pkl.get("metadata")
vehicle_config = optimization_metadata["config"]["vehicle"]

# Get battery capacity for SOC calculations
battery_capacity_kwh = vehicle_config["max_battery_kwh"]

# Collect all historical solutions across generations
all_F = []
for generation in one_pareto_pkl.get("optimization_history", []):
    all_F.extend(generation["F"])  # F contains [time_min, energy_kwh]
all_F = np.array(all_F)

# ==============================================
# NON-DOMINATED SORTING
# ==============================================

# Initialize non-dominated sorting algorithm
nds_filter = NonDominatedSorting()

# Identify global non-dominated solutions from entire optimization history
dominance_fronts = nds_filter.do(all_F, only_non_dominated_front=False)

# Create boolean mask for Pareto-optimal solutions
is_pareto_optimal = np.zeros(len(all_F), dtype=bool)
is_pareto_optimal[dominance_fronts[0]] = True  # First front is non-dominated

# ==============================================
# VISUALIZATION SETUP
# ==============================================

# Initialize plotly figure
fig = go.Figure()

# ==============================================
# TRACE 1: DOMINATED SOLUTIONS (ALL GENERATIONS)
# ==============================================
fig.add_trace(
    go.Scatter(
        x=all_F[~is_pareto_optimal, 0],  # Time in minutes
        y=all_F[~is_pareto_optimal, 1],  # Energy in kWh
        mode="markers",
        name="Dominated Solutions",
        marker=dict(color="lightgray", size=6, opacity=0.4, line=dict(width=0)),
        hovertemplate=(
            "<b>Time</b>: %{x:.1f} min<br>"
            "<b>Energy</b>: %{y:.3f} kWh<br>"
            "<b>Energy</b>: %{y*1000:.0f} Wh<br>"
            "<b>SOC Used</b>: %{customdata:.1%}<br>"
            "<i>Dominated solution</i>"
            "<extra></extra>"
        ),
        customdata=all_F[~is_pareto_optimal, 1] / battery_capacity_kwh,
    )
)

# ==============================================
# TRACE 2: HISTORICAL PARETO FRONT (ALL GENERATIONS)
# ==============================================
fig.add_trace(
    go.Scatter(
        x=all_F[is_pareto_optimal, 0],  # Time in minutes
        y=all_F[is_pareto_optimal, 1],  # Energy in kWh
        mode="markers",
        name="Historical Pareto",
        marker=dict(color="blue", size=8, opacity=0.7, line=dict(width=0)),
        hovertemplate=(
            "<b>Time</b>: %{x:.1f} min<br>"
            "<b>Energy</b>: %{y:.3f} kWh<br>"
            "<b>T/E Ratio</b>: %{customdata[0]:.2f} min/kWh<br>"
            "<b>SOC Used</b>: %{customdata:.1%}<br>"
            "<i>Historical optimal</i>"
            "<extra></extra>"
        ),
        customdata=all_F[is_pareto_optimal, 1] / battery_capacity_kwh,
    )
)

# ==============================================
# TRACE 3: FINAL PARETO FRONT (FROM one_pareto_df)
# ==============================================
if not one_pareto_df.empty:
    fig.add_trace(
        go.Scatter(
            x=one_pareto_df["time_min"],
            y=one_pareto_df["energy_kwh"],
            mode="markers",
            name="Final Pareto",
            marker=dict(
                color="red",
                size=4,
                opacity=1,
                line=dict(color="black", width=1),
            ),
            customdata=np.stack(
                (
                    # Calculate time/energy ratio
                    one_pareto_df["time_min"] / one_pareto_df["energy_kwh"],
                    # Existing SOC calculation
                    one_pareto_df["energy_kwh"] / battery_capacity_kwh,
                ),
                axis=-1,
            ),
        )
    )

# ==============================================
# LAYOUT CONFIGURATION (UNCHANGED)
# ==============================================
fig.update_layout(
    title="Solution Evolution with Dominance Analysis",
    title_x=0.5,
    title_font=dict(size=24),
    xaxis_title="Travel Time [minutes]",
    yaxis_title="Energy Consumption [kWh]",
    legend_title="Solution Types",
    width=1000,
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    hovermode="closest",
    xaxis=dict(
        gridcolor="lightgray",
        zerolinecolor="lightgray",
        showline=True,
        linewidth=2,
        linecolor="black",
        title_standoff=20,
    ),
    yaxis=dict(
        gridcolor="lightgray",
        zerolinecolor="lightgray",
        showline=True,
        linewidth=2,
        linecolor="black",
        title_standoff=20,
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor="rgba(255,255,255,0.8)",
    ),
)

# ==============================================
# EXTREME POINT ANNOTATIONS (UNCHANGED)
# ==============================================
if not one_pareto_df.empty:
    min_time_point = one_pareto_df.loc[one_pareto_df["time_min"].idxmin()]
    min_energy_point = one_pareto_df.loc[one_pareto_df["energy_kwh"].idxmin()]

    fig.add_annotation(
        x=min_time_point["time_min"],
        y=min_time_point["energy_kwh"],
        text="Time-Optimal",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-40,
        font=dict(size=12),
    )

    fig.add_annotation(
        x=min_energy_point["time_min"],
        y=min_energy_point["energy_kwh"],
        text="Energy-Optimal",
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=40,
        font=dict(size=12),
    )

fig.show()
fig.write_image(
    "../reports/figures/pareto_front_dominance_analysis.png",
    width=1000,
    height=600,
    scale=2,
    engine="kaleido",
)

### 3. Running metric

In [13]:
from pymoo.util.running_metric import RunningMetricAnimation

running = RunningMetricAnimation(delta_gen=5, n_plots=3, key_press=False, do_show=True)

for algorithm in result.history:
    running.update(algorithm)

NameError: name 'result' is not defined